In [ ]:
import os
import pandas as pd
import win32com.client
from datetime import datetime
import calendar

def processar_aniversariantes():
    # === CONFIGURAÇÕES ===
    CAMINHO_EXCEL = r"C:\Users\lz6321\OneDrive - grendene.com.br\Documentos\Automações\Automação_Aniversariantes\Aniversariantes.xlsx"
    NOME_ABA = "Sheet1"

    # Configuração dos meses
    MES_CARDS = 3  # Março (pasta onde estão os cartões)
    MES_PROCESSAR = 7  # Julho (mês que queremos criar os eventos)
    ANO = datetime.now().year  # Ano atual

    # Dicionário com nomes dos meses
    MESES = {
        1: "Janeiro", 2: "Fevereiro", 3: "Março", 4: "Abril",
        5: "Maio", 6: "Junho", 7: "Julho", 8: "Agosto",
        9: "Setembro", 10: "Outubro", 11: "Novembro", 12: "Dezembro"
    }

    # Configurar pasta de cards e caminho da marca
    PASTA_CARDS = rf"C:\Users\lz6321\OneDrive - grendene.com.br\Documentos\Automações\Automação_Aniversariantes\Aniversariantes - {MESES[MES_CARDS]}"
    CAMINHO_MARCA = r"C:\Users\lz6321\OneDrive - grendene.com.br\Documentos\Automações\Automação_Aniversariantes\Marca_GovTi.png"
    EMAIL_DESTINO = "carla.vieira@grendene.com.br"

    # Verificar se a pasta do mês existe
    if not os.path.exists(PASTA_CARDS):
        print(f"[AVISO] Pasta para o mês {MESES[MES_CARDS]} não encontrada: {PASTA_CARDS}")
        print("Por favor, verifique o caminho da pasta com os cartões.")
        return

    print(f"[INFO] Usando cartões de {MESES[MES_CARDS]} para criar eventos em {MESES[MES_PROCESSAR]}")

    try:
        # === LER PLANILHA ===
        df = pd.read_excel(CAMINHO_EXCEL, sheet_name=NOME_ABA)
        
        # Filtrar apenas aniversariantes do mês desejado
        df = df[df['MÊS_NUMERO'] == MES_CARDS]
        
        if df.empty:
            print(f"Não há aniversariantes no mês {MES_CARDS}")
            return
            
        # Agrupar por dia para processar cada dia uma única vez
        for dia, grupo in df.groupby('aniversario_natalicio DIA', sort=True):
            try:
                nomes = grupo['Nome Funcionário'].tolist()
                nomes_texto = ";".join(nomes)
                
                print(f"\n[PROCESSANDO] Aniversariantes do dia {dia:02d} (usando cartão de {MES_CARDS:02d} para criar em {MES_PROCESSAR:02d}):")
                print(nomes_texto)
                
                # Inicializar Outlook e acessar calendário
                outlook = win32com.client.Dispatch("Outlook.Application")
                namespace = outlook.GetNamespace("MAPI")
                calendar_folder = namespace.GetDefaultFolder(9)
                
                # Verificar e remover eventos existentes
                data_evento = datetime(ANO, MES_PROCESSAR, dia, 8, 0)  # 8:00
                restricao = f"[Subject]='Aniversariantes do dia' AND [Start] = '{data_evento.strftime('%m/%d/%Y')}'"
                eventos_existentes = calendar_folder.Items.Restrict(restricao)
                
                while eventos_existentes.Count > 0:
                    eventos_existentes.Item(1).Delete()
                    eventos_existentes = calendar_folder.Items.Restrict(restricao)
                
                # Criar novo evento
                appointment = outlook.CreateItem(1)
                
                # Configurar propriedades do evento
                # Criar título com os nomes dos aniversariantes
                if len(nomes) == 1:
                    titulo = f"Aniversário do Contribuinte: {nomes[0]}"
                else:
                    titulo = f"Aniversário dos Contribuintes: {' e '.join(nomes)}"
                
                appointment.Subject = titulo
                appointment.Start = data_evento
                appointment.Duration = 30  # 30 minutos
                appointment.AllDayEvent = False
                appointment.ReminderSet = True
                appointment.ReminderMinutesBeforeStart = 1440  # 24h antes
                appointment.BusyStatus = 0  # Livre
                
                # Configurar corpo do evento com HTML
                corpo_html = f"""<html>
<body>
<p>Olá!</p>
<p>Segue lembrete do aniversário para os colaboradores:<br>
{nomes_texto}.</p>
<p>ATENÇÃO!: No dia do aniversário dos referidos colaboradores deve-se enviar a imagem em anexo na equipe de celebrações no Teams.</p>
<p>Data: {dia:02d}/{MES_PROCESSAR:02d}/{ANO}<br>
Horário: 08:00 às 08:30</p>
<p>Atenciosamente,</p>
<img src="{CAMINHO_MARCA}" style="width:100%;max-width:600px;">
</body>
</html>"""
                
                appointment.HTMLBody = corpo_html
                
                # Anexar marca GovTI
                if os.path.exists(CAMINHO_MARCA):
                    appointment.Attachments.Add(CAMINHO_MARCA)
                else:
                    print("[INFO] Marca GovTI não encontrada")
                
                # Configurar convite
                recipient = appointment.Recipients.Add(EMAIL_DESTINO)
                recipient.Type = 1  # Participante obrigatório
                appointment.MeetingStatus = 1  # Reunião
                
                # Salvar e anexar cartões
                if appointment.Recipients.ResolveAll():
                    appointment.Save()
                    
                    # Anexar cartões
                    anexos = 0
                    arquivos = os.listdir(PASTA_CARDS)
                    for nome in nomes:
                        nome_lower = nome.lower()
                        for arquivo in arquivos:
                            if nome_lower in arquivo.lower():
                                caminho_completo = os.path.join(PASTA_CARDS, arquivo)
                                appointment.Attachments.Add(caminho_completo)
                                anexos += 1
                                break
                    
                    # Enviar convite
                    appointment.Send()
                    print(f"[OK] Evento criado e convite enviado para {EMAIL_DESTINO} com {anexos} anexos.")
                else:
                    print(f"[ERRO] Não foi possível resolver o email {EMAIL_DESTINO}.")
                    
            except Exception as e:
                print(f"[ERRO] Falha ao processar dia {dia:02d}: {str(e)}")
                continue
                
        print("\n[OK] Processamento concluído!")
        
    except Exception as e:
        print(f"[ERRO] Falha ao executar a automação: {str(e)}")

# Executar o processamento
if __name__ == "__main__":
    processar_aniversariantes()

[INFO] Usando cartões de Março para criar eventos em Julho

[PROCESSANDO] Aniversariantes do dia 01 (usando cartão de 03 para criar em 07):
- DENER DAVI SOUSA LIMA
- MARTIM LUIS CREMA
[INFO] Logo do GovTI não encontrada
[OK] Evento criado e convite enviado para carla.vieira@grendene.com.br com 2 anexos.

[PROCESSANDO] Aniversariantes do dia 02 (usando cartão de 03 para criar em 07):
- AFRANIO BIN
- CAMILA DE AZEVEDO OLIVEIRA
[INFO] Logo do GovTI não encontrada
[OK] Evento criado e convite enviado para carla.vieira@grendene.com.br com 2 anexos.

[PROCESSANDO] Aniversariantes do dia 04 (usando cartão de 03 para criar em 07):
- GABRIEL CAMARGO
[INFO] Logo do GovTI não encontrada
[OK] Evento criado e convite enviado para carla.vieira@grendene.com.br com 1 anexos.

[PROCESSANDO] Aniversariantes do dia 08 (usando cartão de 03 para criar em 07):
- JOHNATAS FELIX ZUZA
- PRISCILA MERZONI
[INFO] Logo do GovTI não encontrada
[OK] Evento criado e convite enviado para carla.vieira@grendene.com.br c